## ETL - Extract, Transform and Load

En esta etapa del proyecto, se llevara a cabo el proceso de ETL (extracción, transformación y carga), para poder comenzar con la obtención de valiosa e importante información a partir de los datos. En esta etapa podremos dimensionar la información con la que contamos, las diversas estructuras y formatos de los datos, las relaciones entre los distintos datasets que trabajaremos, para su porterior utilización, y que esta, sea correcta, efectiva y eficaz.

#### Importación de librerias:

In [146]:
import pandas as pd
from io import BytesIO
import pyarrow.parquet as pq
import os
import Utilidades as ut

### Business 📊

*Definimos la ruta del archivo :*

In [3]:
ruta = './Datos/Yelp/business.pkl' 

*Almacenamos el contenido del archivo en un DataFrame:*

In [4]:
with open(ruta, "rb") as file:
    df_business = pd.read_pickle(BytesIO(file.read()))

*Visualizamos el DataFrame:*

In [7]:
df_business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


*Podemos ver que se duplicaron las columnas, asique procedemos a eliminar los duplicados:*

In [8]:
df_business = df_business.loc[:,~df_business.columns.duplicated()]

*Verificamos:*

In [9]:
df_business.head(2)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."


*Normalizamos los nombres de las columnas:*

In [10]:
ut.normalizacion_columnas(df_business)

,Business_Id,Name,Address,City,State,Postal_Code,Latitude,Longitude,Stars,Review_Count,Is_Open,Attributes,Categories,Hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,IN,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,DE,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,AB,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,AB,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


*Filtraremos el DataFrame, y nos quedaremos con uno de menor tamaño, solo con los registros cuyo nombre (columna **Name**) contenga **McDonald**:*

In [22]:
df_mcdonalds = df_business[df_business['Name'].str.contains('McDonald|Mc Donald|Mcdonald|McDonalds|Mc Donalds|Mcdonalds', case=False, na=False)]

*Analizamos los estados donde se encuentran las sucursales de Mc Donald's:*

In [24]:
df_mcdonalds['State'].unique()

array(['PA', 'NJ', 'AZ', 'MO', 'FL', 'CA', 'ID', 'IN', 'LA', 'TN', 'DE',
       'NV', 'AB', 'IL'], dtype=object)

In [25]:
df_mcdonalds.head(2)

,Business_Id,Name,Address,City,State,Postal_Code,Latitude,Longitude,Stars,Review_Count,Is_Open,Attributes,Categories,Hours
193,yM8LlTInbQH4FwWC97lz6w,McDonald's,1919 S Jefferson,St. Louis,PA,63104,38.612495,-90.221942,1.5,100,1,"{'Alcohol': 'u'none'', 'BikeParking': 'True', ...","Fast Food, Restaurants, Food, Burgers, Coffee ...","{'Monday': '0:0-0:0', 'Tuesday': '0:0-0:0', 'W..."
361,aNXw3PkXVt8ANwLyCfcmpg,McDonald's,2333 Welsh Rd,Lansdale,NJ,19446,40.263706,-75.317916,1.5,17,1,"{'WiFi': 'u'free'', 'GoodForKids': 'True', 'Bu...","Restaurants, Fast Food, Burgers, Food, Coffee ...","{'Monday': '6:0-23:0', 'Tuesday': '6:0-23:0', ..."


In [148]:
df_mcdonalds.to_parquet('./Datos/Archivos Filtrados/business_mcdonalds.parquet')

### Checkin 📊

*Definimos la ruta del archivo :*

In [26]:
ruta = './Datos/Yelp/checkin.json'

*Almacenamos el contenido del archivo en un DataFrame:*

In [27]:
df_checkin = pd.read_json(ruta, lines=True)

*Visualizamos la estructura del DataFrame:*

In [28]:
df_checkin.head(2)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."


*Normalizamos los nombres de las columnas del DataFrame:*

In [29]:
ut.normalizacion_columnas(df_checkin)

,Business_Id,Date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."
...,...,...
131925,zznJox6-nmXlGYNWgTDwQQ,"2013-03-23 16:22:47, 2013-04-07 02:03:12, 2013..."
131926,zznZqH9CiAznbkV6fXyHWA,2021-06-12 01:16:12
131927,zzu6_r3DxBJuXcjnOYVdTw,"2011-05-24 01:35:13, 2012-01-01 23:44:33, 2012..."
131928,zzw66H6hVjXQEt0Js3Mo4A,"2016-12-03 23:33:26, 2018-12-02 19:08:45"


*Para poder filtrar los registros, nos quedaremos con las columnas **Name** y **Business_Id** del **df_mcdonalds** (DataFrame que contiene toda la información del comercio):*

In [30]:
mcdonalds = df_mcdonalds[['Name', 'Business_Id']]

*Luego, uniremos ambos DataFrames, **mcdonalds** y **df_checkin**, para quedarnos solo con los registros del comercio de nuestro interes:*

In [31]:
checkin_mcdonalds = mcdonalds.merge(df_checkin, on='Business_Id', how='left')

*Verificamos:*

In [32]:
checkin_mcdonalds.head(2)

,Name,Business_Id,Date
0,McDonald's,yM8LlTInbQH4FwWC97lz6w,"2010-07-05 01:18:34, 2010-07-14 13:22:59, 2010..."
1,McDonald's,aNXw3PkXVt8ANwLyCfcmpg,"2012-03-15 13:38:23, 2012-06-17 00:03:02, 2012..."


*Eliminaremos la columna **Business_Id**, por que no es relevante, ya que ahora tenemos el nombre del comercio:*

In [33]:
checkin_mcdonalds.drop('Business_Id', axis=1, inplace=True)

*Guardamos el DataFrame resultante para su posterior uso:*

In [34]:
checkin_mcdonalds.to_parquet('./Datos/Archivos Filtrados/checkin_mcdonalds.parquet')

### Tip 📊

*Definimos la ruta del archivo :*

In [35]:
ruta = './Datos/Yelp/tip.json' 

*Almacenamos el contenido del archivo en un DataFrame:*

In [36]:
df_tip = pd.read_json(ruta, lines=True)

*Visualizamos la estructura del DataFrame:*

In [39]:
df_tip.head(2)

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0


*Normalizamos los nombres de las columnas:*

In [40]:
ut.normalizacion_columnas(df_tip)

,User_Id,Business_Id,Text,Date,Compliment_Count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


*El DataFrame generado anteriormente, **mcdonalds**, lo uniremos al DataFrame **df_tip** para quedarnos con los tips del comercio de nuestro interes:*

In [42]:
tip_mcdonalds = mcdonalds.merge(df_tip, on='Business_Id', how='left')

*Verificamos:*

In [45]:
tip_mcdonalds.head(2)

,Name,Business_Id,User_Id,Text,Date,Compliment_Count
0,McDonald's,yM8LlTInbQH4FwWC97lz6w,jCLjVfSMbvLkSH2ZMkryZQ,Oatmeal on the go,2011-12-16 13:54:39,0.0
1,McDonald's,yM8LlTInbQH4FwWC97lz6w,6-7HgY0b84ocY9ACESLApA,Awful service.,2015-05-23 17:02:08,0.0


*Eliminaremos la columna **Business_Id**, por que no es relevante, ya que ahora tenemos el nombre del comercio:*

In [46]:
tip_mcdonalds.drop(columns= 'Business_Id', inplace=True)

*Guardamos el DataFrame resultante para su posterior uso:*

In [47]:
tip_mcdonalds.to_parquet('./Datos/Archivos Filtrados/tip_mcdonalds.parquet')

### Review 📊

*Definimos la ruta del archivo :*

In [49]:
ruta = './Datos/Yelp/review.json' 

*Creamos un función que lea el archivo en formato **.json** en fragmentos de 10000 y lo almacene en una lista vacia. Concatenamos los frangmentos de DataFrame en uno solo y guardamos el archivo en formato **.parquet** para que su peso sea menor:*

In [50]:
# Lista vacia para almacenar los fragmentos del DataFrame
dataframes = []

# Tamaño de los fragmentos
tamano_fragmento = 10000

# Leemos el archivo en fragmentos y creamos DataFrames
with open(ruta, 'r') as file:
    for chunk in pd.read_json(file, lines=True, chunksize=tamano_fragmento):
        dataframes.append(chunk)

# Concatenamos los DataFrames en uno solo
df_final = pd.concat(dataframes, ignore_index=True)

# Guardamos el DataFrame en formato Parquet para que su peso sea menor
df_final.to_parquet('review_temporal.parquet')


*Almacenamos el contenido del archivo en un DataFrame, para su posterior transformación:*

In [51]:
df_reviews = pd.read_parquet('review_temporal.parquet')

*Visualizamos la estructura del DataFrame:*

In [52]:
df_reviews.head(2)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18


*Normalizamos los nombres de las columnas del DataFrame:*

In [53]:
ut.normalizacion_columnas(df_reviews)

,Review_Id,User_Id,Business_Id,Stars,Useful,Funny,Cool,Text,Date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


*Uniremos los DataFrames, **mcdonalds** y **df_reviews**, para quedarnos solo con las reseñas del comercio de nuestro interes:*

In [54]:
reviews_mcdonalds = mcdonalds.merge(df_reviews, on= 'Business_Id', how= 'left')

*Verificamos:*

In [55]:
reviews_mcdonalds.head(2)

,Name,Business_Id,Review_Id,User_Id,Stars,Useful,Funny,Cool,Text,Date
0,McDonald's,yM8LlTInbQH4FwWC97lz6w,SB8GmxCcWLkz5hnArAWDyg,6z3PRqgD0ZRzo3JWA2XGiQ,1,0,0,0,If you can't come to work with pride and energ...,2018-01-09 21:23:02
1,McDonald's,yM8LlTInbQH4FwWC97lz6w,F99EHFlmHnt3U6wSdM2KoA,bQ_qlqlZRFgfRlqveB-ILg,2,2,2,0,Not the greatest fast food place to hit at nig...,2010-11-24 21:44:42


*Eliminaremos la columna **Business_Id**, por que no es relevante, ya que ahora tenemos el nombre del comercio:*

In [56]:
reviews_mcdonalds.drop(columns=['Business_Id'], inplace=True)

*Guardamos el DataFrame resultante para su posterior uso:*

In [151]:
reviews_mcdonalds.to_parquet('./Datos/Archivos Filtrados/reviews_mcdonalds.parquet')

### User 📊

*Definimos la ruta del archivo :*

In [59]:
ruta = './Datos/Yelp/user.parquet'

*Almacenamos el contenido del archivo en una variable, **df_user**:*

In [60]:
df_user = pq.read_table(ruta, use_threads=True)

*Lo convertimos a un DataFrame de pandas:*

In [61]:
df_user = df_user.to_pandas()

*Visualizamos la estructura del DataFrame:*

In [62]:
df_user.head(2)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946


In [93]:
df_user.columns

Index(['User_Id', 'Name', 'Review_Count', 'Yelping_Since', 'Useful', 'Funny',
       'Cool', 'Elite', 'Friends', 'Fans', 'Average_Stars', 'Compliment_Hot',
       'Compliment_More', 'Compliment_Profile', 'Compliment_Cute',
       'Compliment_List', 'Compliment_Note', 'Compliment_Plain',
       'Compliment_Cool', 'Compliment_Funny', 'Compliment_Writer',
       'Compliment_Photos'],
      dtype='object')

*Normalizamos los nombres de las columnas del DataFrame:*

In [149]:
ut.normalizacion_columnas(df_user).head(2)

,User_Id,Name,Review_Count,Yelping_Since,Useful,Funny,Cool,Elite,Friends,Fans,...,Compliment_More,Compliment_Profile,Compliment_Cute,Compliment_List,Compliment_Note,Compliment_Plain,Compliment_Cool,Compliment_Funny,Compliment_Writer,Compliment_Photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946


*Guardamos el DataFrame resultante para su posterior uso:*

In [152]:
reviews_mcdonalds.to_parquet('./Datos/Archivos Filtrados/df_user.parquet')

### Metadatos Sitios 📊

*Definimos la ruta del archivo :*

In [64]:
ruta = './Datos/Google Maps/metadata-sitios'

*Leemos los archivos **.json** de la carpeta **metadatos-sitios** , los filtramos quedandonos con aquellos registros cuyo **name** contenga **McDonald** en el nombre del comercio y los unimos, conteniendolos en un DataFrame. Convertimos el DataFrame en un archivo compatible con parquet, y lo guardamos de manera local para su posterior uso:*

In [65]:
#Generamos una lista vacia donde almacenaremos los DataFrames
dfs = []

#Iteramos sobre los archivos del directorio
for filename in os.listdir(ruta):

    #Si el archivo es un JSON, lo leemos y filtramos los registros que contengan la palabra McDonald's
    if filename.endswith('.json'):
        filepath = os.path.join(ruta, filename)
        df = pd.read_json(filepath, lines=True)
        df_filtered = df[df['name'].str.contains(r"\bMcDonald's\b|\bMc Donald's\b", case=False, na=False, regex=True)]
        dfs.append(df_filtered)

#Si se encontraron DataFrames, los concatenamos y guardamos el resultado en formato Parquet
if len(dfs) > 0:
    merged_df = pd.concat(dfs)
    merged_df.reset_index(drop=True, inplace=True)

    # Guardamos el DataFrame en formato Parquet
    merged_df.to_parquet('/Datos/metadatos-sitios-temporal.parquet')
    
else:
    print("No se encontraron archivos JSON en el directorio que cumplieran con la condición.")



*Leemos el archivo generado anteriormente y lo almacenamos en un DataFrame para proceder con las transformaciones del mismo:*

In [70]:
df_sitios = pd.read_parquet('./Datos/metadatos-sitios-temporal.parquet')

*Normalizamos los nombres de las columnas del mismo:*

In [114]:
ut.normalizacion_columnas(df_sitios).head(2)

,Name,Address,Gmap_Id,Description,Latitude,Longitude,Category,Avg_Rating,Num_Of_Reviews,Price,Hours,Misc,State,Relative_Results,Url
0,McDonald's,"McDonald's, 1205 S Main St, Manteca, CA 95336",0x80904101ce001281:0x76db23c5d22346ae,"Classic, long-running fast-food chain known fo...",37.785995,-121.218062,Fast food restaurant Breakfast restaurant Coff...,2.4,48,$,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x8090410018dc2657:0xed7a807ae3eeee6d, 0x8090...",https://www.google.com/maps/place//data=!4m2!3...
1,McDonald's,"McDonald's, 1000 Palisades Center Dr, West Nya...",0x89c2e9cf8e139235:0x24bfb20e9e09f260,"Classic, long-running fast-food chain known fo...",41.097768,-73.955392,Fast food restaurant Breakfast restaurant Coff...,2.2,18,$,None,{'Accessibility': ['Wheelchair accessible rest...,None,"[0x89c2e9e6ef010ddb:0xe923f7207b70d6f9, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...


*Verificamos los valores unicos en la columna **Name** y podemos ver que hay otros comercios que incluyen el nombre Mc Donald's, pero también hay comercios que son los corrrectos, los de nuestro interes, pero contienen cierta descripción o datos de la ubicación en su nombre, por lo que vamos a buscar la forma de filtrarlos:*

In [72]:
df_sitios['Name'].unique()

array(["McDonald's", "McDonald's Studio", "McDonald's Lunch",
       "Mc Donald's Towing of Paw Paw", "Mc Donald's",
       "McDonald's OMG Building", "McDonald's Corporate Office.",
       "Norman McDonald's Country Drive-In",
       "McDonald's Warehouse Corporate office", "McDonald's Cafe & BBQ",
       "Mcdonald's Self Storage", "McDonald's Budget Printing",
       "Peninsula McDonald's Office", "Mc Donald's Kennels",
       "Graviss McDonald's Disc Golf Course", "Mcdonald's Playplace",
       "McDonald's Regional Office", "Mc Donald's RV Park & Car Wash",
       "McDonald's Kennel",
       "Martin Brower L.L.C/McDonald's Distribution Center",
       "Mc Donald's on church ave", "McDonald's - Corporate Office",
       "Mcdonald's Play Area", "Mc Donald's Service Station",
       "Bluemound Rd. at McDonald's",
       "Cabin in the Clouds Christmas Forest (Formerly McDonald's Tree Farm)",
       "McDonald's Transmission Repair", "McDonald's / Ross's",
       "McDonald's HVAC", "McDon

*Podemos ver que los elementos contenidos en nuestra columna **Category**, se encuentran en forma de lista. Procederemos a transformarlos para poder realizar las transformaciones necesarias en esa columna:*

In [73]:
df_sitios['Category'].value_counts()

Category
[Fast food restaurant]                                                                                                                    32
[Corporate office]                                                                                                                        11
[Restaurant]                                                                                                                              10
[Dessert shop]                                                                                                                             5
[Bus stop]                                                                                                                                 2
                                                                                                                                          ..
[Fast food restaurant, Breakfast restaurant, Coffee shop, Hamburger restaurant, Restaurant, Sandwich shop]                                 1
[Fas

*Primero, cambiamos el tipo de dato a **str**:*

In [74]:
df_sitios['Category'] = df_sitios['Category'].astype(str)

*Luego, recorremos cada elemento de la columna, y eliminamos los corchetes y las comillas simples:*

In [77]:
df_sitios['Category'] = df_sitios['Category'].apply(lambda x: x.replace('[','').replace(']','').replace('\'',''))

*Verificamos:*

In [84]:
df_sitios.head(2)

,Name,Address,Gmap_Id,Description,Latitude,Longitude,Category,Avg_Rating,Num_Of_Reviews,Price,Hours,Misc,State,Relative_Results,Url
0,McDonald's,"McDonald's, 1205 S Main St, Manteca, CA 95336",0x80904101ce001281:0x76db23c5d22346ae,"Classic, long-running fast-food chain known fo...",37.785995,-121.218062,Fast food restaurant Breakfast restaurant Coff...,2.4,48,$,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x8090410018dc2657:0xed7a807ae3eeee6d, 0x8090...",https://www.google.com/maps/place//data=!4m2!3...
1,McDonald's,"McDonald's, 1000 Palisades Center Dr, West Nya...",0x89c2e9cf8e139235:0x24bfb20e9e09f260,"Classic, long-running fast-food chain known fo...",41.097768,-73.955392,Fast food restaurant Breakfast restaurant Coff...,2.2,18,$,None,{'Accessibility': ['Wheelchair accessible rest...,None,"[0x89c2e9e6ef010ddb:0xe923f7207b70d6f9, 0x89c2...",https://www.google.com/maps/place//data=!4m2!3...


*Filtramos el DataFrame, quedandonos con aquellos registros que poseen las palabras "Restaurant", "restaurant", "Fast food" y/o "fast-food" en los elementos de la columna **Category**:*

In [80]:
df_sitios_ = df_sitios[df_sitios['Category'].str.contains('restaurant|Fast food|Restaurant|fast-food')]

*Guardamos el DataFrame resultante para su posterior uso:*

In [153]:
df_sitios_.to_parquet('./Datos/Archivos Filtrados/metadatos_mcdonalds.parquet')

*Analizamos aquellos que no contienen ninguno de los juegos de palabras anteriores:*

In [111]:
df_sitios[~df_sitios['Category'].str.contains('restaurant|Fast food|Restaurant|fast-food')].head(2)

,Name,Address,Gmap_Id,Description,Latitude,Longitude,Category,Avg_Rating,Num_Of_Reviews,Price,Hours,Misc,State,Relative_Results,Url
24,McDonald's Studio,"McDonald's Studio, 141 Bridge Ave E, Delano, M...",0x52b4a9af2ca9aa29:0x20efe9b9990af8b3,None,45.041810,-93.788087,Portrait studio,4.9,8,None,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x52b4a9105e951a1d:0x1ac4d8fb0264249d, 0x52b4...",https://www.google.com/maps/place//data=!4m2!3...
28,Mc Donald's Towing of Paw Paw,"Mc Donald's Towing of Paw Paw, 39617 W Red Arr...",0x881741d71056f773:0xde1e96ed0a460024,None,42.211307,-85.933143,Towing service Auto wrecker,3.2,8,None,None,None,None,"[0x88176a4b799092ed:0x10fc20d91d5ecfc9, 0x8817...",https://www.google.com/maps/place//data=!4m2!3...


*Podemos ver claramente, que corresponden a otras categorias totalmente distintas y no pertenecen a la cadena de comidas rapidas de nuestro interes.*

### review-Florida 📊

In [85]:
ruta = './Datos/Google Maps/reviews-estados/review-Florida'

*Leemos los archivos **.json** de la carpeta **review-Florida**. Convertimos el DataFrame en un archivo compatible con parquet, y lo guardamos de manera local para su posterior uso:*

In [101]:
#Generamos una lista vacia donde almacenaremos los DataFrames
dfs = []

#Iteramos sobre los archivos del directorio
for filename in os.listdir(ruta):

    #Si el archivo es un JSON, lo leemos y filtramos los registros que contengan la palabra McDonald's
    if filename.endswith('.json'):
        filepath = os.path.join(ruta, filename)
        df = pd.read_json(filepath, lines=True)
        dfs.append(df)

#Si se encontraron DataFrames, los concatenamos y guardamos el resultado en formato Parquet
if len(dfs) > 0:
    merged_df = pd.concat(dfs)
    merged_df.reset_index(drop=True, inplace=True)

    #Guardamos el DataFrame en formato Parquet
    merged_df.to_parquet('./Datos/review-Florida-temporal.parquet')
    
else:
    print("No se encontraron archivos JSON en el directorio que cumplieran con la condición.")

*Leemos el archivo generado anteriormente y lo almacenamos en un DataFrame para proceder con las transformaciones del mismo:*

In [116]:
df_florida = pd.read_parquet('./Datos/review-Florida-temporal.parquet')

*Normalizamos los nombres de las columnas del mismo:*

In [117]:
df_florida = ut.normalizacion_columnas(df_florida)

In [141]:
df_florida

,User_Id,Name,Time,Rating,Text,Pics,Resp,Gmap_Id
0,1.014719e+20,Julie A. Gerber,1628003250740,1,Update: Their “reply” to my review amounted to...,None,{'text': 'Thank you for the feedback. We would...,0x8893863ea87bd5dd:0x9383ebf973e74abb
1,1.154772e+20,Martin Sheffield,1595031217005,5,He's a knowledgeable doctor but the way he run...,None,{'text': 'Thank you for reaching out. I’m forw...,0x8893863ea87bd5dd:0x9383ebf973e74abb
2,1.018050e+20,Brian Truett,1522924253567,5,"Best doctor I've ever had, I never wait to be ...",None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
3,1.063444e+20,Tina Sun,1467907819586,1,I was told he is a good doctor. I was trying t...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
4,1.008751e+20,James Haynes,1480683415081,5,Takes the time to actually get to know his pat...,None,None,0x8893863ea87bd5dd:0x9383ebf973e74abb
...,...,...,...,...,...,...,...,...
2849995,1.002877e+20,James Rudolph,1617305583696,5,None,None,"{'text': 'Thank you!', 'time': 1617561671234.0}",0x8890966585e36d3f:0x131d47c2c60a8d31
2849996,1.140989e+20,Vincent Alexander,1581817848416,3,None,None,None,0x8890966585e36d3f:0x131d47c2c60a8d31
2849997,1.123448e+20,Brett Owen,1551240010030,5,None,None,None,0x8890966585e36d3f:0x131d47c2c60a8d31
2849998,1.054465e+20,ashly kindle,1521152570004,5,None,None,"{'text': 'Thank you, Ashly', 'time': 151691419...",0x8890966585e36d3f:0x131d47c2c60a8d31


*Uniremos los DataFrames, **df_sitios_** y **df_florida**, para quedarnos solo con las reseñas del comercio de nuestro interes:*

In [142]:
reviews_florida = df_florida.merge(df_sitios, on= 'Gmap_Id')

In [145]:
reviews_florida.head(2)

,User_Id,Name_x,Time,Rating,Text,Pics,Resp,Gmap_Id,Name_y,Address,...,Longitude,Category,Avg_Rating,Num_Of_Reviews,Price,Hours,Misc,State,Relative_Results,Url
0,1.123131e+20,Crystal Pedraza,1609195591445,5,Good food good service,None,None,0x88e81ee8d1ae20e9:0xf34a362c6c772e99,McDonald's,"McDonald's, 13300 Cortez Blvd, Brooksville, FL...",...,-82.506031,Fast food restaurant Breakfast restaurant Coff...,2.2,38,$,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x88e81eea3f689d27:0xade1e6e09892161, 0x88e81...",https://www.google.com/maps/place//data=!4m2!3...
1,1.122551e+20,Jessica Guiliano,1572482113768,1,I am here at 8:30 PM and the only thing my dau...,None,None,0x88e81ee8d1ae20e9:0xf34a362c6c772e99,McDonald's,"McDonald's, 13300 Cortez Blvd, Brooksville, FL...",...,-82.506031,Fast food restaurant Breakfast restaurant Coff...,2.2,38,$,None,{'Accessibility': ['Wheelchair accessible entr...,None,"[0x88e81eea3f689d27:0xade1e6e09892161, 0x88e81...",https://www.google.com/maps/place//data=!4m2!3...


*Guardamos el DataFrame resultante para su posterior uso:*

In [154]:
df_sitios_.to_parquet('./Datos/Archivos Filtrados/reviews_florida_mcdonalds.parquet')